#### 라이브러리 및 환경변수 세팅

In [1]:
import numpy as np
import pandas as pd
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient 
from google.analytics.data_v1beta.types import (
    RunReportRequest, 
    Dimension, 
    Metric,
    OrderBy, 
    DateRange, 
)

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'GCP에서_생성한_json_파일.json'
property_id = 'GA4_속성_ID'
client = BetaAnalyticsDataClient()

### 맞춤 측정기준 만들기
GTM에서 심은 이벤트가 점점 많아지다보면 이벤트 명을 최대한 줄이고,<br>
세세한 부분은 매개변수로 구분하는게 훨씬 이벤트를 관리하기에 효율적인걸 알 수 있다.<br>
이렇게 세팅한 매개변수를 GA4 보고서에서 측정기준으로 설정해서 보려면 맞춤 측정기준을 만들어야한다.<br>
맞춤 측정기준을 세팅하는 방법은 다음과 같다.

- GTM에서 이벤트에 새로운 매개변수 세팅
- GA4 [데이터 표시 > 맞춤 정의]에서 '맞춤 측정기준 만들기' 클릭 후, 세팅한 매개변수 추가
- 1~2일 후 부터 측정 기준으로 사용.
- [GA4 API에서 측정기준](https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema?hl=ko#custom_dimensions)으로 "customEvent:parameter_name"로 설정하여 데이터 불러오기

In [12]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="customEvent:all_event")],
        metrics=[Metric(name="active1DayUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'customEvent:all_event'}, desc=True)], 
        date_ranges=[DateRange(start_date="2024-01-01", end_date="2024-02-01")],
    )

In [42]:
response = client.run_report(request)
data = [] 
for row in response.rows:
    row_data = {dim.name: row.dimension_values[i].value for i, dim in enumerate(response.dimension_headers)}
    row_data.update({metric.name: float(row.metric_values[i].value) for i, metric in enumerate(response.metric_headers)})
    data.append(row_data)
df = pd.DataFrame(data)
df.loc[135,:].reset_index(drop=True)

0    mweb_back_btn
1          43595.0
Name: 135, dtype: object